# 3-Template Matching
in this section we have two main goals :
1) aligning two images :
* mark eyes of each image 

* shift second image based on differences between eyes location

* remove portions of images that are not aligned 

2) use to aligned image two make hybrid image :

* make a Gaussian highpass filter with specific sigma to implement it on near image 

* make a Gaussian lowpass filter with specific sigma to implement it on far image 

* take inverse Fourier transform from both high_frequncy and low_frequency image and add them together.

## aligning images 
to do so , we use `plt.ginput(2)` to get coordinates of eyes in each image by clicking on them . function , then we pass two images and their eye's coordinates to `align(image1, image2, points1, points2)`. this function align images using `np.roll(image, -delta, axis=) `function .

note that because `np.roll` shift start of image to the end of that and vice versa ,after shifting we have to remove unaligned portions of two images 

* original images :
<table><tr>
<td> <img src="res19-near.jpg" alt="Drawing" style="width: 300px;"/> </td>
<td> <img src="res20-far.jpeg" alt="Drawing" style="width: 300px;"/> </td>
</tr></table>


* aligned images( size of the right image changes and shifted a little too ) :

<table><tr>
<td> <img src="res21-near.jpg" alt="Drawing" style="width: 300px;"/> </td>
<td> <img src="res22-far.jpg" alt="Drawing" style="width: 300px;"/> </td>
</tr></table>

 $\color{red}{\text{* note that plt.ginput() doesn't work in jupyter so to examine the Validity of question4's code , run q4.py in any IDE(vscode,pycharm,...)}}$
 
 $\color{red}{\text{* when you run the code of q4.py , first image appears! mark her eyes from left to right and after marking first image's eyes , second image }}$
 $\color{red}{\text{appears. do the same thing for second image. after a few seconds , all of the result will be save }}$


In [7]:
def align(im1, im2, p1, p3):
    im1 = np.array(im1)
    [c1, c2, c3] = im1.shape

    # difference in x position for left eyes of two images
    delta_x = p3[0]-p1[0]
    
    # difference in y position for left eyes of two images
    delta_y = p3[1]-p1[1]

    im2 = np.roll(im2, int(-delta_y), axis=0)  # shift image up/down

    im2 = np.roll(im2, int(-delta_x), axis=1)  # shift image right/left

    if(delta_x > 0):

        im1 = im1[:, :int(c2-delta_x), :]
                                          # crop image after shifting image
        im2 = im2[:, :int(c2-delta_x), :]

    else:

        im1 = im1[:, int(-delta_x):, :]
                                         # crop image after shifting image
        im2 = im2[:, int(-delta_x):, :]

    if(delta_y > 0):

        im1 = im1[:int(c1-delta_y), :, :]
                                         # crop image after shifting image
        im2 = im2[:int(c1-delta_y), :, :]

    else:

        im1 = im1[int(-delta_y):, :, :]
                                        # crop image after shifting image
        im2 = im2[int(-delta_y):, :, :]

    return im1, im2

## make hybrid images
first of all we get Fourier transform of aligned image .(left side is fft of near image and right side is  fft of far image):

<table><tr>
<td> <img src="res23-dft-near.jpg" alt="Drawing" style="width: 400px;"/> </td>
<td> <img src="res24-dft-far.jpg" alt="Drawing" style="width: 400px;"/> </td>
</tr></table>


In [8]:
def fft_cal(image):

    dft = np.fft.fft2(image)

# # apply shift of origin to center of image
    dft_shift = np.fft.fftshift(dft)

    return dft_shift

# ---------------------------------

def fft_inverse_cal(dft):

    ishift = np.fft.ifftshift(dft)

    ifft = np.fft.ifft2(ishift)

    return np.real(ifft)

to make hybrid images first we have to make highpass Gaussian and low pass Gaussian filter .

below are the picture of these filters (left side is high pass and right side is low pass ) 

<table><tr>
<td> <img src="res25-highpass-65.jpg" alt="Drawing" style="width: 300px;"/> </td>
<td> <img src="res26-lowpass-20.jpg" alt="Drawing" style="width: 300px;"/> </td>
</tr></table>

In [9]:
def low_guassian(n1: int, n2: int, sigma):

    c1 = (n1-1)//2

    c2 = (n2-1)//2

    matrix = np.zeros([n1, n2])

    for x in range(0, n1):
        for y in range(0, n2):

            matrix[x, y] = np.exp(-((x-c1)**2+(y-c2)**2)/2/(sigma**2))

    plt.imsave(f'res26-lowpass-{sigma}.jpg', matrix)

    return matrix
# ------------------------------
def high_guassian(n1: int, n2: int, sigma):

    c1 = (n1-1)//2

    c2 = (n2-1)//2

    matrix = np.zeros([n1, n2], dtype='float32')

    for x in range(0, n1):
        for y in range(0, n2):

            matrix[x, y] = np.exp(-((x-c1)**2+(y-c2)**2)/2/(sigma**2))

    matrix = 1-matrix

    plt.imsave(f'res25-highpass-{sigma}.jpg', matrix)

    return matrix

after that we apply filters to fft of images (lowpass filter for far image and highpass filter for near image) .

below are the fft of filtered images (left side is fft of highpassed image and right side is fft of lowpassed image):

<table><tr>
<td> <img src="res27-highpassed.jpg" alt="Drawing" style="width: 400px;"/> </td>
<td> <img src="res28-lowpassed.jpg" alt="Drawing" style="width: 400px;"/> </td>
</tr></table>



In [10]:
def filtering(r, g, b, mask2):

    eq1_r = mask2*r
    eq1_g = mask2*g
    eq1_b = mask2*b

    return eq1_r, eq1_g, eq1_b

# --------------------------------

def mergging(r, g, b, image1):

    merged_image = np.zeros_like(image1)

    r_abs = np.abs(r)
    g_abs = np.abs(g)
    b_abs = np.abs(b)

    merged_image = cv2.merge((r_abs, g_abs, b_abs))

    return merged_image

in the next step we sum highpassed and lowpassed image and plot its dft : 
<table><tr>
<td> <img src="res29-hybrid.jpg" alt="Drawing" style="width: 400px;"/> </td>
</tr></table>


in the last step we get inverse Fourier transform and plot images with different resolutions : 
<table><tr>
<td> <img src="res30-hybrid-near.jpg" alt="Drawing" style="width: 1300px;"/> </td>
<td> <img src="res31-hybrid-far.jpg" alt="Drawing" style="width: 150px;"/> </td>
</tr></table>



In [ ]:
def freq_to_time_domain(r, g, b, image):

    image1 = np.array(image)

    [c1, c2, c3] = image1.shape

    time_domain_image = np.zeros([c1, c2, c3])

    # print(b[350, 350], 'freq to time')

    r_ifft = (fft_inverse_cal(r)).astype('uint8')
    g_ifft = (fft_inverse_cal(g)).astype('uint8')
    b_ifft = (fft_inverse_cal(b)).astype('uint8')

    # print(b_ifft[350, 350], 'freq to time1')

    time_domain_image = mergging(
        r_ifft, g_ifft, b_ifft, image).clip(0, 255)

    return time_domain_image

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image as im

# --------------------------------
# Read first not_aligned image

image1 = cv2.imread("res19-near.jpg")              
image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
image1 = np.array(image1[:, :, :])
[c1, c2, c3] = image1.shape
plt.imshow(image1)
(p1, p2) = plt.ginput(2)                  #marking eyes of the first image
plt.close()


# read second not aligned image
image2 = cv2.imread("res20-far.jpeg")
image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
image2 = np.array(image2)
image2 = cv2.resize(image2, dsize=(c2, c1), interpolation=cv2.INTER_CUBIC)
plt.imshow(image2)
(p3, p4) = plt.ginput(2)                 #marking eyes of the first image
plt.close()

#----------------------------------------- aligning
im1, im2 = align(image1, image2, p1, p3)
im1 = np.array(im1)
im2 = np.array(im2)

# ---------------------------------------- store aligned images
data = im.fromarray(np.uint8(im2))
data.save('res22-far.jpg')  # save the result

data = im.fromarray(np.uint8(im1))
data.save('res21-near.jpg')  # save the result

# -------------------------------

image1 = cv2.imread("res21-near.jpg")
image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
image1_grayscale = cv2.cvtColor(image1, cv2.COLOR_RGB2GRAY)
image1 = np.array(image1[:, :, :])
[c1, c2, c3] = image1.shape

# ------------------------------------

image2 = cv2.imread("res22-far.jpg")
image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
image2_grayscale = cv2.cvtColor(image2, cv2.COLOR_RGB2GRAY)
image2 = np.array(image2[:, :, :])
[c1, c2, c3] = image2.shape

# ------------------------------------dft of first aligned image 

r1, g1, b1 = cv2.split(image1)

r1_dft = fft_cal(r1)
g1_dft = fft_cal(g1)
b1_dft = fft_cal(b1)

# print(b1_dft[350, 350], 'sa')

# ----------------------------------- dft of second aligned image 
r2, g2, b2 = cv2.split(image2)

r2_dft = fft_cal(r2)
g2_dft = fft_cal(g2)
b2_dft = fft_cal(b2)

# -----------------------------------
low = low_guassian(image1.shape[0], image1.shape[1], 20)
plt.imsave('lowpassfilter.jpg', low)

high = high_guassian(image1.shape[0], image1.shape[1], 65)
plt.imsave('highpassfilter.jpg', high)
# ------------------------------------

r_filtered, g_filtered, b_filtered = filtering(r1_dft, g1_dft, b1_dft, high)
image_high = freq_to_time_domain(r_filtered, g_filtered, b_filtered, image1)

r_filtered2, g_filtered2, b_filtered2 = filtering(r2_dft, g2_dft, b2_dft, low)
image_low = freq_to_time_domain(r_filtered2, g_filtered2, b_filtered2, image2)

# -------------------------------------add highpassed and lowpassed image 
final_image = (1*image_high+1*image_low)

